In [2]:
import urllib.request
from spacy import displacy
import spacy
nlp = spacy.load('en')
from spacy.matcher import Matcher

# now import doctable
import sys
sys.path.append('..')
import doctable as dt

In [3]:
# download some example documents which are national security strategy documents
base = 'https://raw.githubusercontent.com/devincornell/intro_to_text_analysis/master/duke_workshop/nss/'
urls = (
    base+'trump_nss.txt',
    base+'obama_nss.txt',
)
texts = [urllib.request.urlopen(url).read().decode('utf-8') for url in urls]
print('trump:', texts[0][:80])
print('obama:', texts[1][:80])

trump: An America that is safe, prosperous, and free at home is an America with the str
obama: Today, the United States is stronger and better positioned to seize the opportun


In [12]:
# make a new doctable object
class NSSPars(dt.DocTable2):
    schema = (
        ('id','integer',dict(primary_key=True, autoincrement=True)),
        ('source', 'string'), # trump or obama - whichever nss document
        ('parid', 'integer'), # paragraph id
        ('sents','pickle'), # contains token parsetrees
        ('ind_src_par', 'index', ('source','parid'),dict(unique=True)),
    )
    def __init__(self,**kwargs):
        super().__init__(schema=self.schema, tabname='nsspars', **kwargs)
        
class NSSParsParser(NSSPars, dt.DocParser):
    def __init__(self, *args, **kwargs):
        
        # create parser and matcher for parsing
        self.nlp = spacy.load('en')
        self.matcher = Matcher(nlp.vocab)
        
        # matches hyphens
        pattern = [{'IS_SPACE':False},{'TEXT':'-'},{'IS_SPACE':False}]
        self.matcher.add('hyphens', None, pattern)
        
        # matches hyphens
        pattern = [{'TEXT':'@'},{'IS_ALPHA':True}]
        self.matcher.add('handles', None, pattern)
        
        # doctable init
        super().__init__(*args, **kwargs)
    
    def insert_document(self, sourcename, text):
        # because distribute_parse takes a list of texts, we wrap in a list but
        # provide a paragraph separator so that it will process paragraphs in
        # parallel
        #doc = self.distribute_parse([text], self.nlp, paragraph_sep='\n\n')[0]
        paragraphs = self.distribute_parse(texts, self.nlp, parsefunc=self.parsetree_tokenize, preprocessfunc=None, 
            paragraph_sep='\n\n', n_cores=4, verbose=False)[0]
        
        valid_sents = list()
        i = 0
        for par in paragraphs:
            if len(par) > 0:
                # filter out sentences with no tokens
                sents = [s for s in par if len(s) > 0]
        
                # insert into table
                self.insert({'source': source, 'sents':sents, 'parid':i})
                i += 1
        
    @classmethod
    def parsetree_tokenize(cls,doc):
        info = {'lemma':cls.lemmainfo, 'prefix':cls.prefixinfo}
        return cls.get_parsetrees(doc, tok_parse_func=None, info_func_map=info, merge_ents=True, 
            spacy_ngram_matcher=cls.matcher, merge_noun_chunks=False)
        
    @staticmethod
    def lemmainfo(tok):
        return tok.lemma_
    @staticmethod
    def prefixinfo(tok):
        return tok.prefix_
    
    @classmethod
    def tokparser(cls,tok):
        return cls.parse_tok(tok, replace_num=True, replace_digit=None, lemmatize=False, normal_convert=None, 
            format_ents=True, ent_convert=None)
    
    @classmethod
    def include_tok(cls,tok):
         return cls.use_tok(tok, filter_whitespace=True, filter_punct=False, filter_stop=False, 
                      filter_digit=False, filter_num=False, filter_all_ents=False, 
                      filter_ent_types=tuple())
    

trump, obama = texts    
db = NSSParsParser()
db.insert_document('trump', trump)
db.insert_document('obama', obama)

Exception during reset or similar
Traceback (most recent call last):
  File "/home/utopia3/dc326/local/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/utopia3/dc326/local/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/home/utopia3/dc326/local/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 530, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 140366570628928 and this is thread id 140364765624064.
Exception closing connection <sqlite3.Connection object at 0x7fa9272b1490>
Traceback (most recent call last):
  File "/home/utopia3/dc326/local/anaconda3/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/uto

AttributeError: type object 'NSSParsParser' has no attribute 'matcher'